In [1]:
using Distributions
using Optim

In [11]:
function generate_data(N)
    delta1, delta2 = -3 , -2
    beta1, beta2 = [1 4], [2 3]
    rho = -1
    lambda = exp(rho)/(1+exp(rho))
    
    Y = zeros(N,2)
    X = zeros(N,2)

    for i=1:N
        x1,x2 = rand(Normal(0.0, 1.0),2)
        e1,e2 = rand(Logistic(),2)
        select = rand(Categorical([lambda, (1-lambda)]))

        #see logit response i.e. definitely 11
        eq = []
        for y_init=[[0 0],[1 1],[0 1],[1 0]]
            y1,y2 = sum(beta1.*[1 x1]) + delta1*y_init[2] + e1 >= 0, sum(beta2.*[1 x2]) +delta2*y_init[1] + e2 >= 0

            if false
                println( "$(1*y1) $(1*y2),$(y_init)" )
                println(sum(beta1.*[1 x1]) + delta1*y_init[2] + e1," ", sum(beta2.*[1 x2]) +delta2*y_init[1] + e2)
                println("-----")
            end

            if (y1==y_init[1]) & (y2==y_init[2])
                append!(eq,[y_init])
            end
        end
        
        y1, y2 = eq[1]
        if size(eq)[1] >= 2
            y1, y2 = eq[select]
        end
        Y[i,:] =[y1 y2]
        X[i,:] =[x1 x2]
    end
    return Y,X
end

Y,X = generate_data(200)

([0.0 1.0; 1.0 0.0; … ; 0.0 1.0; 0.0 1.0], [0.0908777 1.45112; 1.63152 -0.227304; … ; -0.926591 0.533604; 0.191804 1.28394])

In [3]:
function ll_obj(params;xs=0,ys=0)
    n = size(xs)[1]
    k = Integer(size(X)[2]/2) + 1
    p = zeros(2,2,n)

    #solve for the conditional probability firm i does not enter
    #index 1- firm, index2 - firm -i's action
    for a_j=0:1
        util1 = params[1] .+ xs[:,1].*params[2]  .+ a_j.*params[3]
        util2 = params[4] .+ xs[:,2].*params[5]  .+ a_j.*params[6]
        p[1,a_j+1,:] = 1 ./(1 .+ exp.(util1))
        p[2,a_j+1,:] = 1 ./(1 .+ exp.(util2))
    end
    
    lambda = exp(params[7])/(1+exp(params[7]))
 
    #solve for probablity of nash
    mult_eq = max.(( p[1,2,:]  .- p[1,1,:] ) .* (  p[2,2,:] .- p[2,1,:] ),0)
    prob01 = ( p[1,2,:] ) .* ( 1 .- p[2,1,:] ) - (1-lambda).*mult_eq
    prob10 = ( 1 .- p[1,1,:] ) .* ( p[2,2,:] ) - (lambda).*mult_eq
    prob00 = p[1,1,:] .* p[2,1,:]
    prob11 = ( 1 .- p[1,2,:] ) .* ( 1 .- p[2,2,:] )
    
    #compute empirical likelihood
    p00 = (1 .- ys[:,1]).*(1 .- ys[:,2])
    p11 = ys[:,1].*ys[:,2]
    p10 = ys[:,1].* (1 .- ys[:,2])
    p01 = (1 .- ys[:,1]) .* ys[:,2]
    
    ll = p00 .* prob00 + p11 .*prob11 .+ p01 .* prob01 .+ p10 .* prob10

    return -sum(log.(max.(ll,0)))    
end

Y,X = generate_data(200)
ll_obj([1 4 -3 2 3 -2 -2];xs=X,ys=Y)

120.19288391476788

In [4]:
function maximum_likelihood(params;xs=0,ys=0)
    ll_trial(params) = ll_obj(params;xs=xs,ys=ys)
    return optimize(ll_trial, params, LBFGS())
end

result =  maximum_likelihood([1 4 -3 2 3 -2 -2.];xs=X,ys=Y)
print(result.minimizer)
print(result.minimum)

[1.57478 3.59867 -3.43091 3.10359 3.78387 -3.07493 -24.4613]115.09593954251719

In [5]:
function monte_carlo()
    trials = 400
    params_200 = zeros((7,trials))
    params_1000 = zeros((7,trials))
    for i =1:trials
        Y_200, X_200 = generate_data(200)
        trial_200 = maximum_likelihood([1 4 -3 2 3 -2 -1.];xs=X_200,ys=Y_200)
        params_200[:,i] = trial_200.minimizer

        
        Y_1000, X_1000 = generate_data(1000)
        trial_1000 = maximum_likelihood([1 4 -3 2 3 -2 -1.];xs=X_1000,ys=Y_1000)
        params_1000[:,i] = trial_1000.minimizer
    end
    return params_200,params_1000
end

results_200,results_1000 = monte_carlo()

([0.861731 1.03247 … 0.934577 1.02724; 3.44587 3.70457 … 4.26136 5.32979; … ; -1.40316 -1.52616 … -1.03161 -2.83952; -24.3615 -2.48054 … -1.33113 -24.5821], [1.1919 1.05565 … 1.1175 0.784667; 3.82232 4.50251 … 3.83395 3.74923; … ; -2.00019 -2.32804 … -1.74511 -1.90338; -0.415832 -0.465755 … -1.70772 -1.45484])

In [6]:
println(mean(results_200[7,:]))
lambda_200 = exp(mean(results_200[7,:]))/(1 + exp(mean(results_200[7,:])))
println(lambda_200)
var_200 = std(results_200[7,:]) *exp(mean(results_200[7,:]))/(1 + exp(mean(results_200[7,:])))^2

-5.825306419295321
0.002943211430480432


0.03854232712757239

In [7]:
println(mean(results_1000[7,:]))
lambda_1000 = exp(mean(results_1000[7,:]))/(1 + exp(mean(results_1000[7,:])))
println(lambda_1000)
var_200 = std(results_1000[7,:]) *exp(mean(results_1000[7,:]))/(1 + exp(mean(results_1000[7,:])))^2

-3.1620927322257213
0.04061742626874054


0.25581424042030115

In [13]:
function print_results(params_200, params_1000)
       
    params_200[7,:] = exp.(params_200[7,:])./(1 .+ exp.(params_200[7,:]))
    params_1000[7,:] = exp.(params_1000[7,:])./(1 .+ exp.(params_1000[7,:]))
    
    
    println("\\begin{center}")
    println("\\begin{table}")
    println("\\normalsize")
    println("\\begin{tabular}{c|c|c|c|c|c}")
    print("& True & Mean & SD  &  Mean  & SD  \\\\\n")
    print("& &  (200 obs) &  (200 obs) &  (1000 obs) &  (1000 obs)  \\\\\n")
    println("\\hline \\hline")
    param_names = ["\$ \\alpha_1 \$","\$ \\beta_1 \$", "\$ \\Delta_1 \$", 
        "\$ \\alpha_2 \$", "\$ \\beta_2 \$", "\$ \\Delta_2 \$","\$ \\lambda \$" ]
    true_param = ["1", "4", "-3", "2", "3", "-2", "0.2689"]
    for i=1:7
        print("$(param_names[i]) &")
        print("$(true_param[i]) &")
        print(" $(round(mean(params_200[i,:]), digits=4)) &")
        print(" $(round(std(params_200[i,:]), digits=3)) &")
        print(" $(round(mean(params_1000[i,:]), digits=4)) &")
        print(" $(round(std(params_1000[i,:]), digits=3)) \\\\\n")
        if (i ==3) | (i==6)
            println( "\\hline \\hline")
        else
            println( "\\hline")
        end
        
    end
    println("\\end{tabular}")
    println("\\end{table}")
    println("\\end{center}")
end

print_results(results_200[:,:], results_1000[:,:])

\begin{center}
\begin{table}
\normalsize
\begin{tabular}{c|c|c|c|c|c}
& True & Mean & SD  &  Mean  & SD  \\
& &  (200 obs) &  (200 obs) &  (1000 obs) &  (1000 obs)  \\
\hline \hline
$ \alpha_1 $ &1 & 1.0805 & 0.525 & 1.0109 & 0.204 \\
\hline
$ \beta_1 $ &4 & 4.2217 & 0.804 & 4.0242 & 0.275 \\
\hline
$ \Delta_1 $ &-3 & -3.1663 & 0.854 & -3.0288 & 0.315 \\
\hline \hline
$ \alpha_2 $ &2 & 2.0789 & 0.482 & 2.003 & 0.207 \\
\hline
$ \beta_2 $ &3 & 3.1749 & 0.521 & 3.0125 & 0.204 \\
\hline
$ \Delta_2 $ &-2 & -2.1757 & 0.686 & -2.0205 & 0.277 \\
\hline \hline
$ \lambda $ &0.2689 & 0.3345 & 0.321 & 0.2699 & 0.171 \\
\hline
\end{tabular}
\end{table}
\end{center}
